# The first competition on kaggle: Talkingdata

## 1. Important notes

### 1.1 Special situations in this competition

* Big data! There are 184903890 rows in training data(about 7GB). Therefore, lots of methods cannot be used any more and we have to choose a subset of training data to do research.

* Usually single model! Lots of good kernels are only using Lightgbm method. Besides, only Lightgbm could set categorical features parameters. Since those base features have a large amount of unique value, it's impossible to do one-hot encoder for training data. So Lightgbm may be the best choice and I do not have enough time to use other models or do model stacking.

* In this notebook, I will write my summary according to the right process of dealing with this big dataset. Actually, I did not get my scores according to this process, which made me very regretful.

### 1.2 Useful Kernel and my comments

* EDA

[Kernel1](https://www.kaggle.com/yuliagm/talkingdata-eda-plus-time-patterns) : 
The first EDA I have read. It's clear and I could have a basic understanding of data and basic features. What's more, it shows relationship between features and conversion rate.

[Kernel2](https://www.kaggle.com/gopisaran/indepth-eda-entire-talkingdata-dataset) : 
It used a new way to plot the data. Also, some conclusions have been used in my future research, such as total number of Clicks per Day&Hour.

* Feature engineering

[Kernel3](https://www.kaggle.com/nanomathias/feature-engineering-importance-testing) : 
It's one of the most important kernels I have read in this competition. It put forward several kinds of ways of doing feature engineering, which gave me a lot of new ideas to create my own new features.


[Kernel4](https://www.kaggle.com/yuliagm/feature-engineering-idea-generator-numeric) : 
This kernel mainly told us how to decide which new features can be included by ploting relationship between new features and conversion rate. It's useful in feature selection but I saw it near the ending day of this competition. Therefore, I did not try to select features by its way.

* Have scores on LB

[Kernel5](https://www.kaggle.com/bk0000/non-blending-lightgbm-model-lb-0-977)
[Kernel6](https://www.kaggle.com/pranav84/lightgbm-fixing-unbalanced-data-lb-0-9680) : 
Kernel 5&6 mainly added some useful groupby aggregation features(actually those features are all from the first kernel in feature engineering above).


[Kernel7](https://www.kaggle.com/anttip/talkingdata-wordbatch-fm-ftrl-lb-0-9769) : 
It provides new features called delta-time feature and uses hash to create these features. Its method saves lots of time, since Pandas cannot deal with time series efficiently. However, I cannot still totally understand the method called Wordbatch it uses in training data.


[Kernel8](https://www.kaggle.com/asraful70/talkingdata-added-new-features-in-lightgbm/code) : 
This kernel gets the best scores in public kernel that can run on kaggle's server(about 16gb RAM). Features in this kernel are all old features I have seen.

* Other useful kernels

[Kernel9](https://www.kaggle.com/yuliagm/how-to-work-with-big-datasets-on-16g-ram-dask) : 
Since kaggle's server have limited memory, this kernel gives lots of tips that can help us to save memory in our codes. It's very useful for my future research.


[Kernel10](https://www.kaggle.com/nanomathias/bayesian-optimization-of-xgboost-lb-0-9769) : 
Since I have used Bayesian hyperparameter tuning in Houseprice competition, I am very sensitive to those kernel that use it to tun parameter. However, this competition has a very big dataset, which means that it's time-consuming and memory-consuming to tun parameter. Anyway, this kernel gives an idea about how to use Bayesian hyperparameter tuning in this competition and I decide to mark it down.

[Kernel11](https://www.kaggle.com/alexanderkireev/experiments-with-imbalance-nn-arch-9728) : 
It used deep learning model in this competition and I did no have enough time to do research on it, but I will.

Actually, I also read lots of other useful kernels and discussions in this competition. They are very helpful for me. Therefore, it is really worthwhile to spend so much time reading kernels and discussion.

### 1.3 High rank solution

* [6th solution](https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/56283)
Single LightGBM model with 48 features. Training on day <= 8, and validating on both day 9 - hour 4, and day-9, hours 5, 9, 10, 13, 14. Then retraining all data.

* [2nd solution](https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/56328)
About two hundreds features including count features, cumcount features, time-delta features, unique-count features, and which [app/os/channel]s each IP appears in the data. Ensemble three LGB predictions and three NN predictions. 5-fold CV.

* [8th solution](https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/56325)
74 features including count by several groups, NextClicks, TargetEncoding over groups, Statistics (mean/var), duplicate orders. Single LightGBM model and then ensemble 7 models with teamates. Use day 7 for target encoding, day 8+9 for training and last 2.5M rows from train as holdout.

* [4th solution](https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/56243)
Use days 7,8 for training and make predictions for 9th day (hours [4,14]). For test predictions, train on all 7,8,9 and making predictions for the test day (10th).Have around 50 models - mostly lightgbms, but also nns , FMs and some linear models. More related with counts of previous (and next ) days, hours, minutes and seconds of ips,apps,device (and their combinations). They put their code in this [link](https://github.com/CuteChibiko/TalkingData)

* [1st solution](https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/56475)
Day 7 & 8 for training and day 9 for validation, and chose the best number of iterations of LGB. Then train a model on day 7 & 8 & 9 with the obtained number of iterations for creating submission. Train on 646 features with special ways. seven bagged LGB models and a single bagged NN. Great idea to use down-sampling.

* [top 1% solution](https://zhuanlan.zhihu.com/p/36580283)
A solution from Zhihu written in Chinese.

* [28th solution](https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/56368)
Some creative ideas.

## 2. Making your local validation as soos as possible

'Trust local CV'! I often saw these words in this competition. It is really a pity that I realised that at the end of this competition. Due to limited RAM, I chose to use last 2.5M data as my validation set and it's clear that this was not a good idea. For next competition, I will must choose my local CV very carefully before doing feature engineering. That's so important!

Then I will paste some useful validation strategies below. Those ideas really impressed me a lot!

"First, training on day <= 8, and validating on both day 9 - hour 4, and day-9, hours 5, 9, 10, 13, 14.
 Then, retraining on all data using 1.2 times the number of trees found by early stopping in validation.
Using two validation sets was to make sure I was not overfiting to one of them. The hours were selected to match the public and private test data. I also watched the train auc in the early days, discarding features that improved validation but also increased the gap with train a lot. I stopped watching train auc in the last week to speed up things, but last time I checked I had a quite small gap."

"We used days 7,8 for training and we were making predictions for 9th day (hours [4,14]) For test predictions, we were training on all 7,8,9 and making predictions for the test day (10th)"

"阶段一，特征构造的时候只选含public的三个小时，用这部分数据可以粗略地筛除一部分特征，比如跟小时的统计值相关的，一部分跟天相关的，毕竟是一个下采样，而且不连续。所以这一部分完成后，用了大部分特征进入第二阶段。
阶段二，训练集改成第8天，测试集第9天，这样可以保证一天内连续，可以做关于小时的特征，以及连续时序，时间窗上的特征，带来的问题是，day的信息丢失了，然后这部分做完了，特征多了起来。进入第三阶段。
阶段三，就是完整的78训练，9天验证了，根据相关性和特征重要度做进一步的特征筛选，修正模型在数据下采样上的偏差。让模型更鲁棒。以及增加时间粒度的实验。
阶段四，最终提交的时候，根据78训练，9验证的earlystop盲打，轮次变为1.1倍，直接梭哈"


## 3. Feature engineering

Firstly, I have to say I feel regretful that I did not make full use of test_supplement.csv file in this competition. Because lots of people said that they added this file into the whole dataset to do feature engineering and improved their scores a lot. Primarily, the most common features are groupby features and nextclicks features. After reading lots of kernels about feature engineering, I had some basic conclusions.

### 3.1 Feature selection strategies

It's impossible to include all the groupby features and there were a large amount of groupby features that need us to select. Therefore, feature selection strategies are very important in this process. In this part, I basically did feature selection in a right way and I chose features according to feature importance and (maybe) my validation scores. However, some more useful metric need to be considered when I decided which features to include, such as more reliable CV and things like Kernel4 above. Also, I will paste some good feature selection strategies I have seen before below:

"Features were mostly tested by adding them one by one, and keeping them if local validation score improved by at least 0.00005. I also added several of them at once, then removed them one by one to see if validation score decreased."

"whenever a batch of features improve local validation score, keep them
whenever a batch of feature decrease score - I will remove the most import features among this batch according to LightGBM, and re-test
whenever features were not used by LightGBM at all i.e. 0 importance score I will drop them."


### 3.2 Magic features---- next_clicks

Also, I found that next_click features are rare magic features. Other people used it to get a huge boost in public LB(from 0.96xx to 0.97xx) . However,Some traditional ways that calculate nextclicks features(such as 'transform(lambda x: x.diff().shift(-1)).dt.seconds') are very time-consuming. So Kernel7 gave an idea that used hash to do it and I spent some time to do research on its code. Besides, I have not used prev_clicks in my solution because I tried some combinations and they did not behave very well.

Then, I want to paste code that calculate next_click features below, since it really helped me learn a lot:

In [1]:
def add_next_click(df):
    print('extracting click1 feature...')
    D = 2**26
    df['category'] = (df['ip'].astype(str) + "_" + df['app'].astype(str) + "_" + df['device'].astype(str) \
                      + "_" + df['os'].astype(str)).apply(hash) % D
    click_buffer = np.full(D, 3000000000, dtype=np.uint32)
    df['epochtime'] = df['click_time'].astype(np.int64) // 10 ** 9
    next_clicks = []
    for category, time in zip(reversed(df['category'].values), reversed(df['epochtime'].values)):
        next_clicks.append(click_buffer[category] - time)
        click_buffer[category] = time
    del click_buffer
    df['next_click'] = list(reversed(next_clicks))
    df.drop(['category', 'epochtime'], axis=1, inplace=True)
    return (df)

### 3.3 Other new features deserve some research

At beggining of this competition, I really want to create new features by myself. But due to limited RAM and disturbing old features, I did not think about that. After the competition, I read lots of solutions and learnt some new features. Therefore, I want to show them below:

"basefeature，历史CVR，历史转换次数，时间窗（day,hour,quater,half hou,3min）各个维度的统计值（count, unique, max，var），连续点击时间间隔，各个维度的排序，以及一些二阶统计值"

"Ratios like number of clicks per ip, app to number of click per app; Target. This is to capture the leak. I modified the target in train data by sorting is_attributed within group by user, app, and click time. The combination of both ways to capture the leak led to a boost between 0.0004 and 0.0005."

"click count within next one/six hours; average attributed ratio of past clicks "

"lagged target encoding. in day1, this features are missing. for day2, I used target mean in day1. for day3, I used target mean in day2. for day4, I used target mean in day3."

## 4.Hyper parameter tuning and Ensembling

### 4.1 Hyper parameter tuning

Also due to limited RAM and time, I did not do hyper parameter tuning. But I got some useful ideas. 

For example, "The tree model is generally a trade-off between the ability to fit and the generalization performance. Specifically, the tree depth, num_leafs, cosample tree, and subsample are the trade-offs between the four parameters. Then the learning rate can improve the model accuracy."

### 4.2 Ensembling

For me, I did some simple model ensembling at the end of this competition. In the last day, our team have two different kernels that got about 0.9796 and 0.9797. One is mine and another is my teamate's. Both of us have some different features so I decide to blend them. However, I knew that I did not have reliable local CV to do model stacking. Therefore, I just used some basic average ways to blend them. I have put my code named "blending model" on my Github. Finally, I got an improvement from 0.9797 to 0.9800, which made me satisfied because that's the last day of the competition.

Besides, I also read some solutions about model ensembling. The most useful idea is to ensemble some LightGBM models and NN models. However, we need to have a good local CV and I cannot build NN models in the competition. So NN models actually were good methods. I would include it into my future learning plan.

## 5. Some final tips

### 5.1 Some new machine learning terms that I have learnt: 

target encoding, down-sampling, adversarial validatio, pseudo-labelling

### 5.2 New knowledge I will learn

* NN models.
* FFM and FM_FTRL models applied on CTR/CVR problems.
* Some programming skills that help me save RAM.
* Build basic server on cloud platform, such as GCP, AWS, AZURE, Aliyun.